In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

%matplotlib inline

In [53]:
!iconv -f cp1251 -t utf-8 vk_feed.csv -o vk_feed_utf.csv 

iconv: cannot open input file `vk_feed.csv': No such file or directory


In [81]:
data = pd.read_csv('cleaned_data.csv', index_col=0)
data.head()

,text,status,predict_proba,predict,lang
0,ребят краснодар нужны доноры положительная для...,1,1.000000,1,ru
1,волгоград прошу максимальный перепост предыдущ...,1,0.969997,1,ru
2,срочно требуются доноры крови любая цельная кр...,1,1.000000,1,ru
4,анонимно люди крик о помощи срочно а точнее в ...,1,1.000000,1,ru
5,люди крик о помощи срочно а точнее в понедельн...,1,1.000000,1,ru


In [82]:
from langdetect import detect

def detect_language(text):
    try:
        lang = detect(text)
        return lang
    except:
        return '-'

In [83]:
data.groupby(by='lang').count()

,text,status,predict_proba,predict
lang,,,,
ru,26108,26108,26108,26108


In [84]:
data = data[data['lang'] == 'ru']

In [85]:
data.groupby(by='status').count()

,text,predict_proba,predict,lang
status,,,,
0,1836,1836,1836,1836
1,24272,24272,24272,24272


In [86]:
import re
from html2text import html2text

def preprocess(text):
    text = html2text(text)
    text = re.sub('[\W_]', ' ', text)
    text = re.sub('_', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub('[A-Za-z0-9]', '', text) #removing english
    text = text.lower()
    return text

In [87]:
data['text'] = data['text'].apply(preprocess)

In [164]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer

class Preprocessor:
    def __init__(self, stemmer, analyser):
        self.stemmer = stemmer
        self.analyser = analyser
        
    def __call__(self, doc):
        return (self.stemmer.stem(w) for w in self.analyser(doc) if len(w) > 3)
    
preporcessor = Preprocessor(RussianStemmer(), CountVectorizer().build_analyzer())

vectorizer = CountVectorizer(stop_words=stopwords.words('russian'), analyzer=preporcessor)
V = vectorizer.fit_transform(data['text'])

print(V.shape)

(26108, 22264)


In [165]:
X = V
print(X.shape)

y = np.array(data['status'])
print(y.shape)

(26108, 22264)
(26108,)


In [167]:
from sklearn.cross_validation import StratifiedKFold

kfold = StratifiedKFold(y, n_folds=4, shuffle=True)
model = RandomForestClassifier(
    n_estimators=1000, 
    n_jobs=4, 
    verbose=1)

for train_index, test_index in kfold:
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model.fit(X_train, y_train)
    print(model.score(X_test, y_test))

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   22.7s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   54.6s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.7min
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:  2.2min finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    1.7s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    2.1s finished


0.974873601961


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   25.6s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.9min
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:  2.4min finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    1.8s finished


0.974413972729


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    6.9s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   27.5s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.9min
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:  2.3min finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    1.4s finished


0.978091006588


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    5.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   24.7s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   59.4s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.8min
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:  2.2min finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    1.2s


0.975026811705


[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    1.5s finished


In [168]:
from sklearn.ensemble import RandomForestClassifier

model.fit(X, y)
print(model.score(X, y))

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   14.5s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   37.1s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:  1.4min finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    2.3s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    4.1s


0.999846790256


[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    5.2s finished


In [169]:
data['predict_proba'] = 1 - model.predict_proba(X)
data['predict'] = model.predict(X)

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    2.4s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    4.2s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    5.3s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    2.4s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    4.7s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    5.8s finished


In [170]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y, data['predict'])

array([[ 1834,     2],
       [    2, 24270]])

In [171]:
false_negative = data[(data['predict'] == 0) & (data['status'] == 1)]
false_negative

,text,status,predict_proba,predict,lang
1991,нужно больше крови,1,0.349792,0,ru
4433,нужна кровь,1,0.367308,0,ru


In [172]:
false_positive = data[(data['predict'] == 1) & (data['status'] == 0)]
false_positive

,text,status,predict_proba,predict,lang
32854,нужна срочная помощь,0,0.677283,1,ru
33218,максимальной репост,0,0.658854,1,ru


In [186]:
word_vector = vectorizer.transform(['ребят очень нужен донор крови выручите пожалуйта'])
print(model.predict_proba(word_vector)[0][1])

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.2s


0.984


[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    0.3s finished


In [23]:
from dill import dill

with open('model/randomforest/vectorizer.pkl', 'wb') as f:
    dill.dump(vectorizer, f)

with open('model/randomforest/model.pkl', 'wb') as f:
    dill.dump(model, f)